In [ ]:
# Borrar al final
!pip install cache
!python --version
!pip install tensorflow==1.15.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Python 3.7.14
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Librerias
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf # Funciones de redes neuronales
%matplotlib inline
import cifar10 # Base de datos de imagenes para la parte de entrenamiento

print(tf.__version__)

1.15.0


In [ ]:
from utils import load_cifar10_data # Utils se encarga de descargar los datos de entrenamiento
                                    # y separarlos para el entrenamiento
x_train, y_train, x_validate, y_validate, x_test, y_test = load_cifar10_data()
# x - Datos / y - etiquetas
# Train - Los datos de entrenamiento
# Validate - Paso intermedio para comprobar el proceso de entrenamiento
# Test- Prueba final para comprobar el aprendizaje

- Download progress: 100.0%
Download finished. Extracting files.
Done.
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_1
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_2
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_3
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_4
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_5
Loading data: data/CIFAR-10/cifar-10-batches-py/test_batch


In [ ]:
import network, train, utils
from layers import ReluLayer, BinaryFullyConnectedLayer, \
    BinaryConvolutionLayer, BatchNormLayer, MaxPoolingLayer

# Netwrok - Define la red neuronal y sus funciones correspondientes
# Train - Algoritmo de MSA que realiza el entrenamiento
# Layers - Abstracion de las capas, los pesos asignados, funciones forward y backward, funcion Hamiltoniano

In [ ]:
# Red neuronal con las caracteristicas | Input nodos, output nodos, loss function
nn = network.NeuralNetwork(in_size=[None, 32, 32, 3], n_out_classes=10, 
                           loss_func=utils.smooth_hinge_loss)
nn.reset_graph()

# Capas que estan enmedio de la red neuronal 
# Hidden Conv-1
nn.add_layer(BinaryConvolutionLayer(
    out_dim=128, filter_size=3))
nn.add_layer(BatchNormLayer(axes=[0, 1, 2]))
nn.add_layer(ReluLayer())

# Hidden Conv-2
nn.add_layer(BinaryConvolutionLayer(
    out_dim=128, filter_size=3))
nn.add_layer(MaxPoolingLayer())
nn.add_layer(BatchNormLayer(axes=[0, 1, 2]))
nn.add_layer(ReluLayer())

# Hidden Conv-3
nn.add_layer(BinaryConvolutionLayer(
    out_dim=256, filter_size=3))
nn.add_layer(BatchNormLayer(axes=[0, 1, 2]))
nn.add_layer(ReluLayer())

# Hidden Conv-4
nn.add_layer(BinaryConvolutionLayer(
    out_dim=256, filter_size=3))
nn.add_layer(MaxPoolingLayer())
nn.add_layer(BatchNormLayer(axes=[0, 1, 2]))
nn.add_layer(ReluLayer())

# Hidden Conv-5
nn.add_layer(BinaryConvolutionLayer(
    out_dim=512, filter_size=3))
nn.add_layer(BatchNormLayer(axes=[0, 1, 2]))
nn.add_layer(ReluLayer())

# Hidden Conv-6
nn.add_layer(BinaryConvolutionLayer(
    out_dim=512, filter_size=3))
nn.add_layer(MaxPoolingLayer())
nn.add_layer(BatchNormLayer(axes=[0, 1, 2]))
nn.add_layer(ReluLayer())

# Hidden Fc-7
nn.add_layer(BinaryFullyConnectedLayer(
    out_dim=1024))
nn.add_layer(BatchNormLayer(axes=[0]))
nn.add_layer(ReluLayer())

# Hidden Fc-8
nn.add_layer(BinaryFullyConnectedLayer(
    out_dim=1024))
nn.add_layer(BatchNormLayer(axes=[0]))
nn.add_layer(ReluLayer())

# Hidden Fc-9
nn.add_layer(BinaryFullyConnectedLayer(out_dim=10))
nn.add_layer(BatchNormLayer(axes=[0]))

nn.finalize()





Instructions for updating:
Use `tf.cast` instead.

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where









In [ ]:
# Define el vector de entrenamiento
data_train = (x_train, y_train)
opt = train.Trainer(nn, data_train)

In [ ]:
# Hyper-parameters | Tasa de penalizacion & Promedio de error
opt.set_rho(0.5)
opt.set_ema_rates(0.999)

In [ ]:
losses_and_accs_train = []
losses_and_accs_valid = []
losses_and_accs_test = []

# Numero de entrenamientos
n_epochs = 500

for t in range(n_epochs):        
    print('Epoch: ', t)

    # Tamanio de datos | Tasa de aprendizaje es dinamico | Nodos outputs
    opt.train_epoch(batch_size=100, ema_decay=0.98, n_output=10, verbose=True)
    losses_and_accs_train.append(
        opt.loss_and_accuracy((x_train, y_train), max_batch=400, inference=True))
    losses_and_accs_test.append(
        opt.loss_and_accuracy((x_test, y_test), max_batch=400, inference=True))
    losses_and_accs_valid.append(
        opt.loss_and_accuracy((x_validate, y_validate), max_batch=400, inference=True))

# Presenta valor de perdida y el porcentaje de exactitud
    print('Train loss/acc: ', losses_and_accs_train[-1],
          'Test loss/acc: ', losses_and_accs_test[-1])
  
losses_and_accs_train = np.asarray(losses_and_accs_train)
losses_and_accs_valid = np.asarray(losses_and_accs_valid)
losses_and_accs_test = np.asarray(losses_and_accs_test)

Epoch:  0
========= Begin epoch =========
batch_size = 100
EMA rates:
[0.999, 0.999, 0.999, 0.999, 0.999, 0.999, 0.999, 0.999, 0.999]
rho:
[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
Iter: 0 of 450 || Estimated train loss/acc: 1.157259, 0.24
Iter: 45 of 450 || Estimated train loss/acc: 1.176455, 0.21
Iter: 90 of 450 || Estimated train loss/acc: 1.064445, 0.21
Iter: 135 of 450 || Estimated train loss/acc: 0.950343, 0.23
Iter: 180 of 450 || Estimated train loss/acc: 0.944341, 0.19
Iter: 225 of 450 || Estimated train loss/acc: 0.893873, 0.27
Iter: 270 of 450 || Estimated train loss/acc: 0.874758, 0.14
Iter: 315 of 450 || Estimated train loss/acc: 0.891537, 0.17
Iter: 360 of 450 || Estimated train loss/acc: 0.760601, 0.26
Iter: 405 of 450 || Estimated train loss/acc: 0.694233, 0.23


In [ ]:
print('Train: ', opt.loss_and_accuracy((x_train, y_train), inference=True,
                                       max_batch=400))
print('Valid: ', opt.loss_and_accuracy((x_validate, y_validate), inference=True,
                                      max_batch=400))
print('Test: ', opt.loss_and_accuracy((x_test, y_test), inference=True,
                                     max_batch=400))

Train:  (0.8120519351959229, 0.3922222220897675)


KeyboardInterrupt: ignored

In [ ]:
# Considera los maximos valores entre todas las epocas
best_epoch = np.argmax(losses_and_accs_valid[:,1]) + 1
print('Best epoch: ', best_epoch)
print('Train acc: ', losses_and_accs_train[best_epoch-1, 1])
print('Valid acc: ', losses_and_accs_valid[best_epoch-1, 1])
print('Test acc: ', losses_and_accs_test[best_epoch-1, 1])

In [ ]:
# Save data
losses_and_accs = np.concatenate(
    [np.asarray(losses_and_accs_train),
     np.asarray(losses_and_accs_valid),
     np.asarray(losses_and_accs_test)], axis=1)

# Plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10,4))
ax1.semilogy(losses_and_accs[:,0], '-o', label='Train loss')
ax1.semilogy(losses_and_accs[:,2], '-o', label='Valid loss')
ax1.semilogy(losses_and_accs[:,4], '-o', label='Test loss')

ax2.plot(losses_and_accs[:,1], '-o', label='Train acc')
ax2.plot(losses_and_accs[:,3], '-o', label='Valid acc')
ax2.plot(losses_and_accs[:,5], '-o', label='Test acc')

for ax in [ax1,ax2]:
    ax.legend()

ax2.set_ylim(0.1,1)
    
print('Final results: ', losses_and_accs[-1])